In [1]:
using IntervalArithmetic, LinearAlgebra, Serialization

In [2]:
setprecision(1024)

LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))

function enlarge(x::Interval{BigFloat})
    isguaranteed(x) || error("interval is not guaranteed")
    return interval(BigFloat(inf(x), RoundDown), BigFloat(sup(x), RoundUp))
end

function Base.inv(P::Bidiagonal{Interval{Float64}, Vector{Interval{Float64}}})
    if P.uplo == 'U'
        C = -P.ev./P.dv[1:end-1]
        invC = UpperTriangular(zeros(eltype(P), size(P)))
        for i in 1:size(P)[1]
            invC[i,i:end] = cumprod([one(eltype(P)); C[i:end]])
        end
        return UpperTriangular(invC ./ P.dv')
    else
        return LowerTriangular(inv(P')')
    end
end    

function Base.:*(D::Diagonal, v::Vector)
    return diag(D).*v
end

function Base.:*(D::Diagonal, A::Matrix)
    return diag(D).*A
end

function Base.:*(A::Matrix, D::Diagonal)
    return A.*diag(D)'
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        B = A'A
        Λ̄, V̄ = eigen(Symmetric(mid.(B)))
        Λ = inv(interval.(V̄))*B*interval.(V̄)
        all(isguaranteed.(Λ)) || error("matrix not guaranteed")
        σ̄ = sup(sqrt(maximum(abs.(diag(Λ) + interval(-1,1)*[sum(abs.(Λ[i,1:i-1]))+sum(abs.(Λ[i,i+1:end])) for i=1:size(B)[1]]))))
        σ̲ = inf(norm(A⁻¹*interval.(V̄)[:,end])/norm(interval.(V̄)[:,end]))
        return interval(σ̲, σ̄)
    end
end

op_norm (generic function with 1 method)

In [3]:
n = 3500
setprecision(8192)
b₁ = deserialize("b1_k4")
C_α = deserialize("C_alpha")
θ = deserialize("theta")
setprecision(precision(mid.(b₁)))
b₀ = interval(BigFloat, 0.0)
b = zeros(Interval{BigFloat},n+4)
b[1:2] = [b₀, b₁]
for k = 2:n+3
    b[k+1] = interval(4)+(interval(BigFloat, k-1)/b[k])-b[k]-b[k-1]
end
setprecision(1024)
b = b[2:end]
a = enlarge.(sqrt.(b))
b = enlarge.(b);

In [4]:
α = interval.(Float64, interval.(collect(1:2:n+1))./a[1:2:n+1])
β = interval.(Float64, a[1:2:n+1].*a[2:2:n+2].*a[3:2:n+3])
A = Bidiagonal(α,  β[1:end-1], :U)
A⁻¹ = Matrix(interval.(Float64, inv(A)));

In [5]:
C₁₂ = interval(1)/(C_α*sqrt(interval(1)-θ^2))*β[end]*norm(A⁻¹[:,end])
C₂₂ = interval(1)/(C_α*(interval(1)-θ));

In [6]:
C̄ₚ₁ = op_norm(A⁻¹)^2;

In [7]:
supCₚ₁ = op_norm([interval(sqrt(sup(C̄ₚ₁))) C₁₂/interval(n+1)^interval(3//4);
            interval(0) C₂₂/interval(n+1)^interval(3//4)])^2;

In [8]:
C₁ = norm([C₁₂, C₂₂]);

In [9]:
Cₚ₁ = interval(inf(C̄ₚ₁), sup(supCₚ₁));

In [10]:
α = interval.(Float64, interval.(collect(2:2:n))./a[2:2:n])
β = interval.(Float64, a[2:2:n].*a[3:2:n+1].*a[4:2:n+2])
A = Bidiagonal(α,  β[1:end-1], :U)
A⁻¹ = Matrix(interval.(Float64, inv(A)));

In [11]:
C₁₂ = interval(1)/(C_α*sqrt(interval(1)-θ^2))*β[end]*norm(A⁻¹[:,end])
C₂₂ = interval(1)/(C_α*(interval(1)-θ));

In [12]:
C̄ₚ₀ = op_norm(A⁻¹)^2;

In [13]:
supCₚ₀ = op_norm([interval(sqrt(sup(C̄ₚ₀))) C₁₂/interval(n)^interval(3//4);
            interval(0) C₂₂/interval(n)^interval(3//4)])^2;

In [14]:
C₀ = norm([C₁₂, C₂₂]);

In [15]:
Cₚ₀ = max(interval(inf(C̄ₚ₀), sup(supCₚ₀)));

In [16]:
C = max(C₁,C₀)

[1.22238, 1.2224]_com

In [17]:
Cₚ = max(Cₚ₁, Cₚ₀)

[33.58, 33.5801]_com

In [21]:
diam(Cₚ)/2

2.25370360595889e-7

In [25]:
serialize("CP1", Cₚ₁)
serialize("CP0", Cₚ₀)
serialize("CP", Cₚ)